In [1]:
import numpy as np
import pandas as pd
import random
import os
import copy
import torch

from itertools import product
from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, log_loss, r2_score, mean_squared_error
from sklearn.decomposition import PCA
from pytorch_tabnet.tab_model import TabNetRegressor

from torch.utils.tensorboard import SummaryWriter

In [2]:
def fix_random(seed):
    torch.manual_seed(seed)
    random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.benchmark = False
    torch.backends.cudnn.deterministic = True
fix_random(42)

df= pd.read_csv('ml-25m/dataset.csv')

In [11]:
pca_t= False

In [12]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [13]:
X = df.drop(['rating'], axis=1).to_numpy()
y = df['rating'].to_numpy()

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state= 42)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state= 42)

y_train= y_train.reshape(-1, 1)
y_val= y_val.reshape(-1, 1)

print("Numvber of train set: ", X_train.shape[0])
print("Numvber of validation set: ", X_val.shape[0])
print("Numvber of test set: ", X_test.shape[0])

Numvber of train set:  8841
Numvber of validation set:  2211
Numvber of test set:  2764


In [14]:
if pca_t== True:
    pca = PCA(n_components=100)
    pca.fit(X_train)
    X_train = pca.transform(X_train)
    X_val = pca.transform(X_val)
    X_test = pca.transform(X_test)
    print("PCA is applied")
else:
    print("PCA is not applied")

PCA is not applied


In [15]:
batch_size = [256]
n_epochs = [200]
n_d= [16, 32, 64]
n_a= [16, 32, 64]
n_steps= [3, 6, 9]
n_indipendent= [2, 3]

params= list(product(batch_size, n_epochs, n_d, n_a, n_steps, n_indipendent))
comb= len(params)
print("Number of combinations: ", comb)

Number of combinations:  54


In [16]:
def get_model(n_d, n_a, n_steps, n_indipendent):
    model = TabNetRegressor(
        n_d=n_d,
        n_a=n_a,
        n_steps=n_steps,
        n_independent=n_indipendent
    )
    return model

In [17]:
best_mse= float('inf')
best_model= None
best_params= None
iter = 0

for batch_size, n_epochs, n_d, n_a, n_steps, n_indipendent in params:
    iter += 1
    print("Iteration: ", {iter}, "of ", {comb})
    print(f"Configuration: batch_size={batch_size}, n_epochs={n_epochs}, n_d={n_d}, n_a={n_a}, n_steps={n_steps}, n_indipendent={n_indipendent}")

    model= get_model(n_d, n_a, n_steps, n_indipendent)
    log= f'batch_size={batch_size}, n_epochs={n_epochs}, n_d={n_d}, n_a={n_a}, n_steps={n_steps}, n_indipendent={n_indipendent}'

    if pca_t== True:
        if os.path.exists(f'risultati/TabNet/pca/'+log):
            print("Model already exists")
            continue
        writer = SummaryWriter(f'risultati/TabNet/pca/batc_size={batch_size}, n_epochs={n_epochs}, n_d={n_d}, n_a={n_a}, n_steps={n_steps}, n_indipendent={n_indipendent}')
    else:
        if os.path.exists(f'risultati/TabNet/no_pca'+log):
            print("Model already exists")
            continue
        writer = SummaryWriter(f'risultati/TabNet/no_pca/batc_size={batch_size}, n_epochs={n_epochs}, n_d={n_d}, n_a={n_a}, n_steps={n_steps}, n_indipendent={n_indipendent}')

    model.fit(
        X_train=X_train,
        y_train=y_train,
        eval_set=[(X_val, y_val)],
        eval_name=["mse"],
        patience= 10,
        batch_size= batch_size,
        virtual_batch_size= batch_size,
    )

    y_pred= model.predict(X_test)
    mse= mean_squared_error(y_test, y_pred)
    r2= r2_score(y_test, y_pred)

    writer.add_hparams(
        {"batch_size": batch_size, "n_epochs": n_epochs, "n_d": n_d, "n_a": n_a, "n_steps": n_steps, "n_indipendent": n_indipendent},
        {"hparam/mse": mse, "hparam/r2": r2},
    )
    print("MSE: ", mse)
    print("R2: ", r2)

    if mse < best_mse:
        best_mse= mse
        best_model= copy.deepcopy(model)
        best_params= (batch_size, n_epochs, n_d, n_a, n_steps, n_indipendent)
        print("Best model updated")
    writer.close()


Iteration:  {1} of  {54}
Configuration: batch_size=256, n_epochs=200, n_d=16, n_a=16, n_steps=3, n_indipendent=2


/opt/anaconda3/lib/python3.9/site-packages/pytorch_tabnet/abstract_model.py:75: UserWarning: Device used : cpu
  warnings.warn(f"Device used : {self.device}")


epoch 0  | loss: 2.75338 | mse_mse: 1.11096 |  0:00:08s
epoch 1  | loss: 0.25636 | mse_mse: 0.94962 |  0:00:15s
epoch 2  | loss: 0.13335 | mse_mse: 0.55284 |  0:00:21s
epoch 3  | loss: 0.09458 | mse_mse: 0.20337 |  0:00:28s
epoch 4  | loss: 0.07569 | mse_mse: 0.1739  |  0:00:34s
epoch 5  | loss: 0.06468 | mse_mse: 0.11133 |  0:00:42s
epoch 6  | loss: 0.06199 | mse_mse: 0.10089 |  0:00:49s
epoch 7  | loss: 0.05848 | mse_mse: 0.06966 |  0:00:56s
epoch 8  | loss: 0.05502 | mse_mse: 0.06349 |  0:01:03s
epoch 9  | loss: 0.0459  | mse_mse: 0.08529 |  0:01:10s
epoch 10 | loss: 0.04407 | mse_mse: 0.05721 |  0:01:16s
epoch 11 | loss: 0.03955 | mse_mse: 0.03838 |  0:01:23s
epoch 12 | loss: 0.03719 | mse_mse: 0.03606 |  0:01:30s
epoch 13 | loss: 0.02884 | mse_mse: 0.02944 |  0:01:35s
epoch 14 | loss: 0.02851 | mse_mse: 0.03698 |  0:01:40s
epoch 15 | loss: 0.02471 | mse_mse: 0.02086 |  0:01:45s
epoch 16 | loss: 0.01993 | mse_mse: 0.01758 |  0:01:51s
epoch 17 | loss: 0.02205 | mse_mse: 0.02083 |  0

/opt/anaconda3/lib/python3.9/site-packages/pytorch_tabnet/callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)


MSE:  0.005460564424748714
R2:  0.9753658111065461
Best model updated
Iteration:  {2} of  {54}
Configuration: batch_size=256, n_epochs=200, n_d=16, n_a=16, n_steps=3, n_indipendent=3


...

epoch 0  | loss: 5.33034 | mse_mse: 1.0771  |  0:00:19s
epoch 1  | loss: 1.15561 | mse_mse: 1.08828 |  0:00:38s
epoch 2  | loss: 0.45227 | mse_mse: 0.99779 |  0:01:03s
epoch 3  | loss: 0.39081 | mse_mse: 0.51411 |  0:01:18s
epoch 4  | loss: 0.16672 | mse_mse: 0.19283 |  0:01:31s
epoch 5  | loss: 0.10369 | mse_mse: 0.09511 |  0:01:45s
epoch 6  | loss: 0.08609 | mse_mse: 0.11242 |  0:01:58s
epoch 7  | loss: 0.07179 | mse_mse: 0.10234 |  0:02:12s
epoch 8  | loss: 0.06691 | mse_mse: 0.10293 |  0:02:25s
epoch 9  | loss: 0.07547 | mse_mse: 0.07586 |  0:02:46s
epoch 10 | loss: 0.07591 | mse_mse: 0.08591 |  0:03:05s
epoch 11 | loss: 0.05533 | mse_mse: 0.07223 |  0:03:24s
epoch 12 | loss: 0.0516  | mse_mse: 0.09196 |  0:03:46s
epoch 13 | loss: 0.04886 | mse_mse: 0.05374 |  0:03:59s
epoch 14 | loss: 0.04665 | mse_mse: 0.05373 |  0:04:14s
epoch 15 | loss: 0.03934 | mse_mse: 0.04778 |  0:04:27s
epoch 16 | loss: 0.05359 | mse_mse: 0.04216 |  0:04:41s
epoch 17 | loss: 0.03738 | mse_mse: 0.05281 |  0

/opt/anaconda3/lib/python3.9/site-packages/pytorch_tabnet/callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)


MSE:  0.011998575981528264
R2:  0.9458709458967645
Iteration:  {52} of  {54}
Configuration: batch_size=256, n_epochs=200, n_d=64, n_a=64, n_steps=6, n_indipendent=3


/opt/anaconda3/lib/python3.9/site-packages/pytorch_tabnet/abstract_model.py:75: UserWarning: Device used : cpu
  warnings.warn(f"Device used : {self.device}")


epoch 0  | loss: 5.05045 | mse_mse: 2.1659  |  0:00:22s
epoch 1  | loss: 0.86124 | mse_mse: 1.67751 |  0:00:47s
epoch 2  | loss: 0.30259 | mse_mse: 1.02914 |  0:01:06s
epoch 3  | loss: 0.17475 | mse_mse: 0.22367 |  0:01:20s
epoch 4  | loss: 0.14516 | mse_mse: 0.11173 |  0:01:34s
epoch 5  | loss: 0.10874 | mse_mse: 0.14066 |  0:01:54s
epoch 6  | loss: 0.10183 | mse_mse: 0.18075 |  0:02:08s
epoch 7  | loss: 0.14449 | mse_mse: 0.08997 |  0:02:22s
epoch 8  | loss: 0.10333 | mse_mse: 0.13406 |  0:02:40s
epoch 9  | loss: 0.08248 | mse_mse: 0.07275 |  0:03:01s
epoch 10 | loss: 0.08294 | mse_mse: 0.11135 |  0:03:21s
epoch 11 | loss: 0.06936 | mse_mse: 0.0762  |  0:03:43s
epoch 12 | loss: 0.07015 | mse_mse: 0.09923 |  0:04:02s
epoch 13 | loss: 0.07066 | mse_mse: 0.082   |  0:04:17s
epoch 14 | loss: 0.06688 | mse_mse: 0.10087 |  0:04:31s
epoch 15 | loss: 0.06227 | mse_mse: 0.0674  |  0:04:45s
epoch 16 | loss: 0.08215 | mse_mse: 0.07616 |  0:04:59s
epoch 17 | loss: 0.05546 | mse_mse: 0.053   |  0

In [ ]:
print("Best model: ", best_model)
print("Best MSE: ", best_mse)
print("R2:", r2_score(y_test, best_model.predict(X_test)))

Best MSE:  0.008822745268496674
Best model:  TabNetRegressor(n_d=16, n_a=32, n_steps=3, gamma=1.3, cat_idxs=[], cat_dims=[], cat_emb_dim=1, n_independent=3, n_shared=2, epsilon=1e-15, momentum=0.02, lambda_sparse=0.001, seed=0, clip_value=1, verbose=1, optimizer_fn=<class 'torch.optim.adam.Adam'>, optimizer_params={'lr': 0.02}, scheduler_fn=None, scheduler_params={}, mask_type='sparsemax', input_dim=100, output_dim=1, device_name='auto', n_shared_decoder=1, n_indep_decoder=1)
R2: 0.9601980387745401


In [ ]:
#save best model
if pca_t== True:
    best_model.save_model(f"risultati/TabNet/best_model_PCA.csv")
else:
    best_model.save_model(f"risultati/TabNet/best_model_no_PCA.csv")

Successfully saved model at risultati/TabNet/best_model_PCA.csv.zip


'risultati/TabNet/best_model_PCA.csv.zip'